In [ ]:
import urbanpy as up
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
manta_limits = up.download.nominatim_osm('Manta, Ecuador', 1)
manta_limits.plot()

In [ ]:
manta_hexs = up.geom.gen_hexagons(9, manta_limits)

In [ ]:
manta_hexs.plot(cmap='viridis')

In [ ]:
manta_hexs.shape

In [ ]:
ecuador_pop = up.download.hdx_fb_population('ecuador', 'full')

In [ ]:
ecuador_pop_elderly = up.download.hdx_fb_population('ecuador', 'elderly')

In [ ]:
ecuador_pop.head()

In [ ]:
type(ecuador_pop)

In [ ]:
plt.scatter(x=ecuador_pop.longitude, y=ecuador_pop.latitude)

In [ ]:
manta_pop = up.geom.filter_population(ecuador_pop, manta_limits)

In [ ]:
manta_pop_elderly = up.geom.filter_population(ecuador_pop_elderly,
                                              manta_limits)

In [ ]:
type(manta_pop)

In [ ]:
manta_pop.head()

In [ ]:
manta_pop.plot('population_2020')

In [ ]:
manta_hexs_pop = up.geom.merge_shape_hex(manta_hexs, manta_pop,
                                         {'population_2020': 'sum'})

In [ ]:
manta_pop_elderly.head()

In [ ]:
manta_hexs_pop_elderly = up.geom.merge_shape_hex(manta_hexs, 
                                                 manta_pop_elderly,
                                                 {'population': 'sum'})

In [ ]:
manta_hexs_pop_elderly.head()

In [ ]:
manta_hexs_pop.head()

In [ ]:
manta_hexs_pop.plot('population_2020', legend=True)

In [ ]:
manta_hexs_pop_elderly.plot('population', legend=True)

In [ ]:
food_facilities = up.download.overpass_pois(bounds=manta_hexs.total_bounds, facilities='food')

In [ ]:
food_facilities.head()

In [ ]:
ax = manta_limits.plot()
food_facilities.plot(ax=ax, color='k')

In [ ]:
centroids = manta_hexs.centroid

In [ ]:
manta_hexs['lon'] = centroids.x
manta_hexs['lat'] = centroids.y

In [ ]:
dist, ind = up.utils.nn_search(food_facilities[['lat', 'lon']].values,
                               manta_hexs[['lat', 'lon']].values)

In [ ]:
manta_hexs['dist_nn'] = dist 

In [ ]:
manta_hexs['ind_nn'] = ind 

In [ ]:
manta_hexs.plot('dist_nn')

Asuminedo que la velocidad de una persona caminando es 5km/h

In [ ]:
# minutos que demoraria una persona caminando para ir al lugar de venta de alimentos + cercano
manta_hexs['dur_nn'] = (manta_hexs['dist_nn'] / 5) * 60

In [ ]:
cuts, labels = up.utils.create_duration_labels(manta_hexs['dur_nn'])

In [ ]:
manta_hexs['dur_nn_bins'] = pd.cut(manta_hexs['dur_nn'], cuts, labels=labels)

In [ ]:
manta_hexs = manta_hexs.merge(manta_hexs_pop[['hex', 'population_2020']],
                              left_on='hex', right_on='hex')

In [ ]:
manta_hexs.query('population_2020 > 0').plot('dur_nn_bins', legend=True, cmap='viridis')

In [ ]:
# start server
up.routing.start_osrm_server('ecuador', 'south-america')

In [ ]:
# Distancia y duración del viaje a pie
distance_duration = manta_hexs.progress_apply(
    lambda row: up.routing.osrm_route(
        origin=row.geometry.centroid, 
        destination=food_facilities.iloc[row['ind_nn']]['geometry'],
        profile='foot'
    ),
    result_type='expand',
    axis=1,
)

In [ ]:
manta_hexs['distance_to_food_facility'] =  distance_duration[0] / 1000 # meters to km
manta_hexs['duration_to_food_facility'] = distance_duration[1] / 60 # seconds to minutes

In [ ]:
fig = up.plotting.choropleth_map(manta_hexs, 'duration_to_food_facility')
                                 

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()

In [ ]:
manta_hexs['duration_to_food_facility_bins'] = pd.cut(manta_hexs['duration_to_food_facility'], cuts, labels=labels)

In [ ]:
fig = up.plotting.choropleth_map(
    manta_hexs, 'duration_to_food_facility_bins',
    title='Acceso a instalaciones de venta de alimento',
    color_discrete_sequence=px.colors.sequential.Plasma_r, 
    category_orders={'duration_to_food_facility_bins': labels}, 
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()

## Identificacion de zonas

In [ ]:
pop_filter = manta_hexs['population_2020'] > 300

In [ ]:
dur_filter = manta_hexs['duration_to_food_facility'] > 30

In [ ]:
ax = manta_limits.plot()
manta_hexs[pop_filter & dur_filter].plot(ax=ax, color='red', alpha=0.5)